In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
import os
import gc
import torch
from sklearn.model_selection import train_test_split
import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

# get all files' names in derictory
os.chdir("D:\Data\Blurring\clean")
clean_name = []
for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        clean_name.append(name)

os.chdir("D:\Data\Blurring\\blur")
blur_name = []
for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        blur_name.append(name)

In [2]:
X_clean = np.zeros(shape=(1, 481, 720, 3))

for name in clean_name:
    img_new = Image.open("D:\Data\Blurring\clean/" + name)
    arr_new = np.asarray(img_new, dtype='uint8')
    X_clean = np.append(X_clean, arr_new[np.newaxis,:], axis=0)
    
X_blur = np.zeros(shape=(1, 481, 720, 3))

for name in blur_name:
    img_new = Image.open("D:\Data\Blurring\\blur/" + name)
    arr_new = np.asarray(img_new, dtype='uint8')
    X_blur = np.append(X_blur, arr_new[np.newaxis,:], axis=0)
    
del clean_name, blur_name, arr_new
gc.collect()

0

In [3]:
X_clean.shape, X_blur.shape

((125, 481, 720, 3), (125, 481, 720, 3))

In [4]:
X_clean = X_clean[1:]
X_blur = X_blur[1:]
X_clean.shape

(124, 481, 720, 3)

In [5]:
X = np.concatenate((X_clean, X_blur), axis=0)

del X_clean, X_blur 
gc.collect()

60

In [8]:
X = torch.tensor(X)
X = X / 255
X.shape

torch.Size([248, 481, 720, 3])

In [9]:
X = X.permute(0, 3, 1, 2)
X.shape

torch.Size([248, 3, 481, 720])

In [10]:
a, b = torch.ones((124,1)), torch.zeros((124, 1))
y = torch.cat((a,b))
y.shape

torch.Size([248, 1])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), \
                                                    y.numpy(), \
                                                    shuffle=True,\
                                                    test_size=.1)
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [12]:
def shapes(lists):
    for el in lists:
        print(el.shape)
        
shapes([X_train, X_test, y_train, y_test])

torch.Size([223, 3, 481, 720])
torch.Size([25, 3, 481, 720])
torch.Size([223, 1])
torch.Size([25, 1])


In [13]:
del X, y
gc.collect()

68

In [21]:
class BlurNet(torch.nn.Module):
    
    def __init__(self):
        super(BlurNet, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=3)
        self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, padding=0)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = torch.nn.Conv2d(in_channels=16, out_channels=8, kernel_size=5, padding=0)
        self.pool4 = torch.nn.MaxPool2d(kernel_size=3, stride=3)
        self.conv5 = torch.nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, padding=0)
        self.pool5 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = torch.nn.Linear(4 * 11 * 18, 140)
        self.fc2 = torch.nn.Linear(140, 40)
        self.fc3 = torch.nn.Linear(40, 1)

        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool3(self.relu(self.conv3(x)))
        x = self.pool4(self.relu(self.conv4(x)))
        x = self.pool5(self.relu(self.conv5(x)))

        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        x = self.sigmoid(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        
        return x     
    
blurnet = BlurNet()
blurnet.to('cuda: 0')

BlurNet(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(3, 3))
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (fc1): Linear(in_features=792, out_features=140, bias=True)
  (fc2): Linear(in_features=140, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=1, bias=True)
)

In [22]:
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(blurnet.parameters(), lr=1.0e-3)

In [23]:
X_train = X_train.to('cuda: 0')
y_train = y_train.to('cuda: 0')

In [24]:
batch_size = 31

for epoch in range(300):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        
        optimizer.zero_grad()
        
        batch_indexes = order[start_index: start_index + batch_size]
        X_batch = X_train[batch_indexes].to('cuda: 0', dtype=torch.float)
        y_batch = y_train[batch_indexes].to('cuda: 0', dtype=torch.float)
        
        y_pred = blurnet.forward(X_batch.float())

        loss_val = loss(y_pred, y_batch)
        loss_val.backward()
        optimizer.step()
    print("Epoch {} OK.".format(epoch))
    #blurnet.forward(X_train[1].float())
    

Epoch 0 OK.
Epoch 1 OK.
Epoch 2 OK.
Epoch 3 OK.
Epoch 4 OK.
Epoch 5 OK.
Epoch 6 OK.
Epoch 7 OK.
Epoch 8 OK.
Epoch 9 OK.
Epoch 10 OK.
Epoch 11 OK.
Epoch 12 OK.
Epoch 13 OK.
Epoch 14 OK.
Epoch 15 OK.
Epoch 16 OK.
Epoch 17 OK.
Epoch 18 OK.
Epoch 19 OK.
Epoch 20 OK.
Epoch 21 OK.
Epoch 22 OK.
Epoch 23 OK.
Epoch 24 OK.
Epoch 25 OK.
Epoch 26 OK.
Epoch 27 OK.
Epoch 28 OK.
Epoch 29 OK.
Epoch 30 OK.
Epoch 31 OK.
Epoch 32 OK.
Epoch 33 OK.
Epoch 34 OK.
Epoch 35 OK.
Epoch 36 OK.
Epoch 37 OK.
Epoch 38 OK.
Epoch 39 OK.
Epoch 40 OK.
Epoch 41 OK.
Epoch 42 OK.
Epoch 43 OK.
Epoch 44 OK.
Epoch 45 OK.
Epoch 46 OK.
Epoch 47 OK.
Epoch 48 OK.
Epoch 49 OK.
Epoch 50 OK.
Epoch 51 OK.
Epoch 52 OK.
Epoch 53 OK.
Epoch 54 OK.
Epoch 55 OK.
Epoch 56 OK.
Epoch 57 OK.
Epoch 58 OK.
Epoch 59 OK.
Epoch 60 OK.
Epoch 61 OK.
Epoch 62 OK.
Epoch 63 OK.
Epoch 64 OK.
Epoch 65 OK.
Epoch 66 OK.
Epoch 67 OK.
Epoch 68 OK.
Epoch 69 OK.
Epoch 70 OK.
Epoch 71 OK.
Epoch 72 OK.
Epoch 73 OK.
Epoch 74 OK.
Epoch 75 OK.
Epoch 76 OK.
Epoch 77 

In [26]:
predict = blurnet.forward(X_test.float().to('cuda:0'))

In [44]:
print(y_test.flatten().data)
print(predict.round().flatten().data)

tensor([0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 1., 1.])
tensor([0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 1., 1.], device='cuda:0')


tensor([0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 1., 1.], device='cuda:0')

In [55]:
y_test = y_test.to('cuda: 0')

In [56]:
(predict - y_test) ** 2

tensor([[5.6554e-09],
        [4.8532e-10],
        [1.0027e-10],
        [6.0226e-10],
        [4.3265e-08],
        [4.9694e-10],
        [8.4256e-11],
        [9.5629e-10],
        [4.3636e-10],
        [2.1928e-10],
        [1.0027e-10],
        [2.5523e-10],
        [8.4256e-11],
        [6.7618e-09],
        [5.5146e-10],
        [2.3288e-10],
        [1.0027e-10],
        [1.0267e-10],
        [5.5948e-10],
        [1.4496e-10],
        [3.6148e-10],
        [7.9903e-08],
        [3.0536e-10],
        [8.4256e-11],
        [2.2204e-10]], device='cuda:0', grad_fn=<PowBackward0>)

In [ ]:
y_test